In [2]:
data_path = '/home/oscar/Desktop/Exjobb/Data/ieee_supplement/Hyperspectral_Grids/Salinas/'

In [22]:
import numpy as np
import visdom
import utils
import seaborn as sns
from datasets import get_dataset, HyperX
import os
from sklearn import preprocessing
from sklearn.decomposition import PCA
import random

In [4]:
vis = visdom.Visdom()

Setting up a new session...


In [5]:
label_values = ["Undefined", "Brocoli_green_weeds_1", "Brocoli_green_weeds_2", "Fallow",
                        "Fallow_rough_plow", "Fallow_smooth", "Stubble",
                        "Celery", "Grapes_untrained", "Soil_vinyard_develop",
                        "Corn_senesced_green_weeds", "Lettuce_romaine_4wk", "Lettuce_romaine_5wk",
                        "Lettuce_romaine_6wk", "Lettuce_romaine_7wk", "Vinyard_untrained",
                        "Vinyard_vertical_trellis"]

palette = None
rgb_bands = (43, 21, 11)

if palette is None:
    # Generate color palette
    palette = {0: (0, 0, 0)}
    for k, color in enumerate(sns.color_palette("hls", len(label_values) - 1)):
        palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype='uint8'))
invert_palette = {v: k for k, v in palette.items()}

def convert_to_color(x):
    return utils.convert_to_color_(x, palette=palette)
def convert_from_color(x):
    return utils.convert_from_color_(x, palette=invert_palette)

In [6]:
#train_patches = np.zeros(int((len(os.listdir(data_path + '/salinas_fold_0/'))-3)/2))
#train_gt = np.copy(train_patches)

train_patches = []
train_gt = []

for i in (range(int((len(os.listdir(data_path + '/salinas_fold_0/'))-3)/2))):
    #train_patches[i] = np.load(data_path + '/salinas_fold_0/patch_{}.npy'.format(i))
    #train_gt[i] = np.load(data_path + '/salinas_fold_0/patch_{}_gt.npy'.format(i))
    
    train_patches.append(np.load(data_path + '/salinas_fold_0/patch_{}.npy'.format(i)))
    train_gt.append(np.load(data_path + '/salinas_fold_0/patch_{}_gt.npy'.format(i)))

test = np.load(data_path + '/salinas_fold_0/test.npy')
test_gt = np.load(data_path + '/salinas_fold_0/test_gt.npy') 

ValueError: Cannot take a larger sample than population when 'replace=False'

In [12]:
mask = np.ones_like(train_gt)
print(np.count_nonzero(mask))
for i in range(len(mask)):
    mask[i, train_gt[i] == 0] = 0
print(np.count_nonzero(mask))
p,x,y = np.nonzero(mask)
print(str(p) + ' ' + str(x) + '' + str(y))
p_u, x_u, y_u = np.nonzero(mask==0)

5940
4902
[ 0  0  0 ... 26 26 26] [ 0  0  0 ... 21 21 21][0 1 2 ... 7 8 9]


In [13]:
gt_np = np.array(train_gt)
train_np = np.array(train_patches)

In [24]:
idx = set()

while len(idx) < 0.05*np.prod(gt_np.shape):
    p = np.random.randint(gt_np.shape[0])
    x = np.random.randint(gt_np.shape[1])
    y = np.random.randint(gt_np.shape[2])
    if (p,x,y) not in idx:
        idx.add((p,x,y))

In [38]:
idx_np = np.array(list(idx))
idx_np

array([[22,  6,  2],
       [ 8,  1,  7],
       [ 0, 10,  4],
       [23,  3,  7],
       [23, 18,  6],
       [15, 11,  2],
       [23, 14,  4],
       [ 8, 12,  6],
       [21, 10,  3],
       [14,  9,  8],
       [13, 10,  4],
       [25,  1,  6],
       [11,  8,  0],
       [ 4, 12,  7],
       [15, 14,  6],
       [21, 11,  8],
       [26, 20,  0],
       [15, 21,  7],
       [16,  7,  1],
       [19, 15,  0],
       [15,  0,  0],
       [11,  8,  2],
       [ 7, 11,  7],
       [ 3,  0,  7],
       [ 4,  8,  4],
       [ 7, 13,  5],
       [ 1, 13,  4],
       [ 3,  8,  0],
       [ 0,  5,  3],
       [19,  2,  9],
       [25,  4,  1],
       [ 5, 15,  9],
       [12,  4,  6],
       [ 9, 18,  4],
       [20,  0,  3],
       [ 9,  6,  3],
       [ 2, 17,  2],
       [26, 15,  2],
       [ 9, 14,  4],
       [ 9, 19,  7],
       [23,  6,  5],
       [18,  3,  1],
       [ 6, 11,  4],
       [ 9,  1,  5],
       [21,  0,  9],
       [ 5, 17,  3],
       [ 3,  9,  7],
       [18,  

In [42]:
train_np.shape

(27, 22, 10, 204)

In [227]:
ignored_labels = [0]
patch_size = 5

pad_width = patch_size // 2

train_np = np.pad(train_np, ((0,0), (pad_width, pad_width), (pad_width, pad_width), (0,0)))
gt_np = np.pad(gt_np, ((0,0), (pad_width, pad_width), (pad_width, pad_width)))

In [243]:
mask = np.ones_like(gt_np)
for l in ignored_labels:
    mask[gt_np == l] = 0
patch_labeled, x_labeled, y_labeled = np.nonzero(mask)

for l in ignored_labels:
    patch_unlabeled, x_unlabeled, y_unlabeled = np.nonzero(mask==l)

p = patch_size // 2

indices_labeled = np.array([(p_l, x_l, y_l) for p_l, x_l, y_l in zip(patch_labeled, x_labeled, y_labeled) if x_l >= p and x_l < train_np.shape[1] - p and y_l >= p and y_l < train_np.shape[2] - p])
labels = [gt_np[p_l, x_l, y_l] for p_l, x_l, y_l in indices_labeled]

indices_unlabeled = np.array([(p_u, x_u, y_u) for p_u, x_u, y_u in zip(patch_unlabeled, x_unlabeled, y_unlabeled) if x_u >= p and x_u < train_np.shape[1] - p and y_u >= p and y_u < train_np.shape[2] - p])

indices_labeled_shuffle = np.copy(indices_labeled)
indices_unlabeled_shuffle = np.copy(indices_unlabeled)

np.random.shuffle(indices_labeled_shuffle)
np.random.shuffle(indices_unlabeled_shuffle)

In [352]:
p, x, y = indices_labeled[0]

data = train_np[p, x-pad_width:x - pad_width + patch_size, y-pad_width:y-pad_width+patch_size]

label = gt_np[p, x-pad_width:x - pad_width + patch_size, y-pad_width:y-pad_width+patch_size]
labels[0]

15

In [294]:
reshape = train_np.reshape(np.prod(train_np.shape[:3]), np.prod(train_np.shape[3:]))
scaler = preprocessing.MinMaxScaler()
reshape = scaler.fit_transform(reshape)
img_list = reshape.reshape(train_np.shape)

test = np.asarray(test, dtype='float32')
data_test = test.reshape(np.prod(test.shape[:2]), np.prod(test.shape[2:]))
data_test = scaler.transform(data_test)
test_patch = data_test.reshape(test.shape)

In [330]:
np.max((test - np.min(train_np))/(np.max(train_np) - np.min(train_np)))

1.1002865

In [362]:
data_train = np.array([train_np[p_l, x_l, y_l] for p_l, x_l, y_l in zip(patch_labeled, x_labeled, y_labeled) if x_l >= p and x_l < train_np.shape[1] - p and y_l >= p and y_l < train_np.shape[2] - p])

pca = PCA(n_components=11)
pca.fit(data_train)

PCA(copy=True, iterated_power='auto', n_components=11, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [147]:
nalepa_classes = np.zeros(len(label_values))

for i in range(len(train_gt)):
    for c in range(len(nalepa_classes)):
        amount = np.count_nonzero(train_gt[i] == c)
        nalepa_classes[c] += amount

In [111]:
print(nalepa_classes)

[1038.  222.  370.    0.    0.  151.    0.    0. 1411.  220.  837.  100.
   24.   41.  187. 1310.   29.]


In [114]:
print(np.sum(nalepa_classes[1:]))

4902.0


In [364]:
np.count_nonzero(gt_np) + np.count_nonzero(test_gt)

54129

In [127]:
np.shape(np.pad(train_patches, ((0,0), (2,2), (2,2), (0,0))))

(27, 26, 14, 204)

In [22]:
utils.display_predictions(convert_to_color(gt_5), vis)

In [23]:
utils.display_dataset(patch_5, gt_5, rgb_bands, label_values, palette, vis)

Image has dimensions 22x10 and 204 channels


In [7]:
img, gt, label_values, ignored_labels, rgb_bands, palette = get_dataset('Salinas', target_folder='/home/oscar/Desktop/Exjobb/Data/')

In [46]:
img.shape[1]

217

In [16]:
patch_size = 5

In [91]:
x = 0
y = 0

patch_list = []

while y in range(img.shape[1]):
    while x in range(img.shape[0]):
        if y + patch_size <= img.shape[1]: 
            if x + patch_size <= img.shape[0]:
                patch_list.append(img[x:x+patch_size, y:y+patch_size, :])
        '''
            else: 
                patch_list.append(img[x:, y:y+patch_size, :])
        else:
            if x + patch_size <= img.shape[0]:
                patch_list.append(img[x:x+patch_size, y:, :])
            else: 
                patch_list.append(img[x:, y:, :])
        '''
        x += patch_size
    y += patch_size
    x = 0

In [64]:
x = 0
y = 0

gt_list = []

while y in range(gt.shape[1]):
    while x in range(gt.shape[0]):
        if y + patch_size <= gt.shape[1]: 
            if x + patch_size <= gt.shape[0]:
                gt_list.append(gt[x:x+patch_size, y:y+patch_size])
            #else: 
                #gt_list.append(gt[x:, y:y+patch_size])
        #else:
            #if x + patch_size <= gt.shape[0]:
                #gt_list.append(gt[x:x+patch_size, y:])
            #else: 
                #gt_list.append(gt[x:, y:])
        x += patch_size
    y += patch_size
    x = 0

In [65]:
shape_list = []

for i in range(len(gt_list)):
    shape_list.append(np.shape(gt_list[i]))

In [72]:
gt_list[1][3,3]

0

In [35]:
utils.display_dataset(patch_list[453], gt_list[453], rgb_bands, label_values, palette, vis)

Image has dimensions 5x5 and 204 channels


In [32]:
utils.display_predictions(convert_to_color(gt_list[453]), vis)

In [49]:
gt_list[453][3,3]

8

In [86]:
class_list = np.zeros(len(label_values))
gt_center = np.zeros(len(gt_list))

for i in range(len(gt_list)):
    c = gt_list[i][3,3]
    class_list[c] += 1
    gt_center[i] = c

In [87]:
print(class_list)

[2237.   77.  150.   81.   51.  100.  159.  147.  452.  254.  129.   42.
   76.   36.   41.  286.   68.]


In [113]:
print(np.sum(class_list[1:]))

2149.0


In [88]:
np.argmax(np.bincount(gt_list[352].flatten()))

8

In [89]:
class_major_list = np.zeros(len(label_values))
gt_major = np.zeros(len(gt_list))

for i in range(len(gt_list)):
    amount = np.bincount(gt_list[i].flatten())
    c = np.argmax(amount)
    class_major_list[c] += 1
    gt_major[i] = c

In [90]:
print(class_major_list)

[2229.   82.  148.   80.   57.  103.  152.  142.  452.  251.  131.   42.
   77.   36.   43.  290.   71.]


In [112]:
print(np.sum(class_major_list[1:]))

2157.0


21.599999999999998